<a href="https://colab.research.google.com/github/pythonioncoder/Fit-Form-AI/blob/main/Prototype_2_Electric_Boogalooo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **STEP ONE**

In [ ]:
!pip install -q mediapipe==0.10.0

Make sure a copy of the **Fit Form AI Resources** folder is in your personal Drive or at the relevant directory before the next step

Folder can be found here: https://drive.google.com/drive/folders/1RFTh9ed50zVZaIyutPCo6-VTEQanqE4_?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_path = '/content/drive/MyDrive'

if input("Are you using your local drive without any subfolders? (Y / N)\n") != 'Y': # Don't include the Fit Form AI Resources folder itself
  root_path = input("Enter path to the Fit Form AI Resources folder:\n")

# **STEP TWO**

In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

import numpy as np
import cv2 as cv

from IPython.display import display, Javascript, Image, clear_output, HTML
from google.colab.output import eval_js
import html
from base64 import b64decode, b64encode
import PIL
import io

import matplotlib.pyplot as plt
import time
import os
import glob
import csv
import warnings

In [ ]:
# Class that is able to conviently store all poselandmarker data
# Input a poselandmarker dstruct and it outputs the x y or z of any specific body part
# lm_dstruct.nose.x
class lm_dstruct:
  def __init__(self, pll):
    self.left_shoulder = pll[11]
    self.right_shoulder = pll[12]
    self.left_elbow = pll[13]
    self.right_elbow = pll[14]
    self.left_wrist = pll[15]
    self.right_wrist = pll[16]
    self.left_thumb = pll[21]
    self.right_thumb = pll[22]
    self.left_hip = pll[23]
    self.right_hip = pll[24]
    self.left_knee = pll[25]
    self.right_knee = pll[26]
    self.left_ankle = pll[27]
    self.right_ankle = pll[28]
    self.left_heel = pll[29]
    self.right_heel = pll[30]
    self.left_foot_index = pll[31]
    self.right_foot_index = pll[32]
    self.threshold = .66

  # Returns the angle formed by any three points, with pointb as the center
  def angle(self, pointa, pointb, pointc):
    warnings.filterwarnings('ignore')

    pointax = pointa.x - pointb.x
    pointay = pointa.y - pointb.y
    pointaz = pointa.z - pointb.z
    pointcx = pointc.x - pointb.x
    pointcy = pointc.y - pointb.y
    pointcz = pointc.z - pointb.z
    lineba = np.sqrt((pointax**2) + (pointay**2) + (pointaz**2))
    linebc = np.sqrt((pointcx**2) + (pointcy**2) + (pointcz**2))
    lineac = np.sqrt(((pointax - pointcx)**2) + ((pointay - pointcy)**2) + ((pointaz - pointcz)**2))
    return np.arccos(((lineba**2) + (linebc**2) - (lineac**2)) / (2 * lineba * linebc)) * 180 / np.pi # use law of cos to find angle

  # Feedback function
  # Opens the csv for the selected exercise
  # Calculates the angles the exercise requires (determines necessity if weight value signals its needed)
  # Compares the calculated angle with the min-max of the exercise
  def feedback(self, exercise):
    with open(root_path + f'/Fit Form AI Resources/CSV Files/{exercise}.csv') as anglefile:
      anglereader = csv.reader(anglefile)
      for row in anglereader:
        # First check to see if you should calculate the angle
        if row[3] == '1':
          # Then figure out what angle you need to calculate
          # Arm, Grip, Back, Arms by Side, Leg, Hip Angle, Calf
          if row[0] == 'Arm':
            right_vis = (self.right_shoulder.visibility + self.right_elbow.visibility + self.right_wrist.visibility) / 3
            left_vis = (self.left_shoulder.visibility + self.left_elbow.visibility + self.left_wrist.visibility) / 3
            if right_vis > left_vis: # Finds the better side and uses that for angles
              if right_vis < self.threshold:
                return "Right Arm Isn't Visible"
              working_angle = self.angle(self.right_shoulder, self.right_elbow, self.right_wrist)
            else:
              if left_vis < self.threshold:
                return "Left Arm Isn't Visible"
              working_angle = self.angle(self.left_shoulder, self.left_elbow, self.left_wrist)

            if working_angle < int(row[1]): # See if the angle is within bounds
              return "Try Straightening Your Arm Somewhat"
            if working_angle > int(row[2]):
              return "Try Bending Your Elbow More"

          elif row[0] == 'Wrist Angle':
            right_vis = (self.right_elbow.visibility + self.right_wrist.visibility + self.right_thumb.visibility) / 3
            left_vis = (self.left_elbow.visibility + self.left_wrist.visibility + self.left_thumb.visibility) / 3
            if right_vis > left_vis:
              if right_vis < self.threshold:
                return "Right Wrist Isn't Visible"
              working_angle = self.angle(self.right_elbow, self.right_wrist, self.right_thumb)
            else:
              if left_vis < self.threshold:
                  return "Left Wrist Isn't Visible"
              working_angle = self.angle(self.left_elbow, self.left_wrist, self.left_thumb)

            if working_angle < int(row[1]):
              return "Try Straightening Your Wrist Somewhat"
            if working_angle > int(row[2]):
              return "Try Bending Your Wrist More"


          elif row[0] == 'Hip Angle':
            right_vis = (self.right_shoulder.visibility + self.right_hip.visibility + self.right_knee.visibility) / 3
            left_vis = (self.left_shoulder.visibility + self.left_hip.visibility + self.left_knee.visibility) / 3
            if right_vis > left_vis:
              if right_vis < self.threshold:
                return "Right Side of Body Isn't Visible"
              working_angle = self.angle(self.right_shoulder, self.right_hip, self.right_knee)
            else:
              if left_vis < self.threshold:
                return "Left Side of Body Isn't Visible"
              working_angle = self.angle(self.left_shoulder, self.left_hip, self.left_knee)

            if working_angle < int(row[1]):
              return "Try Straightening Your Hip Somewhat"
            if working_angle > int(row[2]):
              return "Try Bending Over More"

          elif row[0] == 'Arms by Side':
            right_vis = (self.right_elbow.visibility + self.right_shoulder.visibility + self.right_hip.visibility) / 3
            left_vis = (self.left_elbow.visibility + self.left_shoulder.visibility + self.left_hip.visibility) / 3
            if right_vis > left_vis:
              if right_vis < self.threshold:
                return "Right Arm and Hip Isn't Visible"
              working_angle = self.angle(self.right_elbow, self.right_shoulder, self.right_hip)
            else:
              if left_vis < self.threshold:
                return "Left Arm and Hip Isn't Visible"
              working_angle = self.angle(self.left_elbow, self.left_shoulder, self.left_hip)

            if working_angle < int(row[1]):
              return "Try Spreading Your Arms Out More"
            if working_angle > int(row[2]):
              return "Try Bringing Your Arms Somewhat Closer To Your Body"

          elif row[0] == 'Leg':
            right_vis = (self.right_hip.visibility + self.right_knee.visibility + self.right_ankle.visibility) / 3
            left_vis = (self.left_hip.visibility + self.left_knee.visibility + self.left_ankle.visibility) / 3
            if right_vis > left_vis:
              if right_vis < self.threshold:
                return "Right Leg Isn't Visible"
              working_angle = self.angle(self.right_hip, self.right_knee, self.right_ankle)
            else:
              if left_vis < self.threshold:
                return "Left Leg Isn't Visible"
              working_angle = self.angle(self.left_hip, self.left_knee, self.left_ankle)

            if working_angle < int(row[1]):
              return "Try Straightening Your Legs More"
            if working_angle > int(row[2]):
              return "Try Bending Your Legs More"

          elif row[0] == 'Calf':
            right_vis = (self.right_ankle.visibility + self.right_heel.visibility + self.right_foot_index.visibility) / 3
            left_vis = (self.left_ankle.visibility + self.left_heel.visibility + self.left_foot_index.visibility) / 3
            if right_vis > left_vis:
              if right_vis < self.threshold:
                return "Right Foot Isn't Visible"
              working_angle = self.angle(self.right_ankle, self.right_heel, self.right_foot_index)
            else:
              if left_vis < self.threshold:
                return "Left Foot Isn't Visible"
              working_angle = self.angle(self.left_ankle, self.left_heel, self.left_foot_index)

            if working_angle < int(row[1]):
              return "Try Pointing Your Feet More (Tiptoe position)"
            if working_angle > int(row[2]):
              return "Try Flattening Your Feet More"

      return "Good Form"

# **STEP THREE**

In [ ]:
model_path = root_path + f'/Fit Form AI Resources/pose_landmarker_{input("What model to use? ")}.task'

What model to use? heavy


If you would like to save a video of the live camera feed, type **'Y'** below. If not, **'N'** or leave blank.

In [ ]:
record = False
vid_path = ''
if input("Would you like to record and save the video you create below? (Y / N)\n") == 'Y':
  record = True
  vid_path = root_path + '/Fit Form AI Resources/Videos/'

In [ ]:
BaseOptions = python.BaseOptions # Basic Options Object
PoseLandmarker = vision.PoseLandmarker # Pose Landmarker Model Object
PoseLandmarkerOptions = vision.PoseLandmarkerOptions # Overall Options Object
VisionRunningMode = vision.RunningMode # Mode Object (Image, Video, Livestream)
PoseLandmarkerResult = vision.PoseLandmarkerResult # Stores result from model
out_video = None
if record:
  out_video = cv.VideoWriter(vid_path + time.asctime(time.localtime()) + '.mp4',cv.VideoWriter_fourcc(*'DIVX'), 24, (640, 480))
landmarks_list = []
feedback_list = [""]
selected_exercise = input("What exercise are you tracking?\n")


# Create a pose landmarker instance with the live stream mode:
def print_result(result: PoseLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
  if result.pose_landmarks:
    annotated_image = np.copy(output_image.numpy_view())
    landmarks_list.append(result.pose_landmarks[0]) # Adds landmark data for pic to list
    for landmark in result.pose_landmarks[0]:
      pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
      pose_landmarks_proto.landmark.extend(
          [landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in result.pose_landmarks[0]])
      solutions.drawing_utils.draw_landmarks(
          annotated_image,
          pose_landmarks_proto,
          solutions.pose.POSE_CONNECTIONS,
          solutions.drawing_styles.get_default_pose_landmarks_style())

    # print("Frame", timestamp_ms)
    working_dstruct = lm_dstruct(landmarks_list[-1])
    feedback_temp = working_dstruct.feedback(selected_exercise)
    if feedback_list[-1] != feedback_temp:
      print(feedback_temp)
    feedback_list.append(feedback_temp)
    if record:
      out_video.write(annotated_image)


options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result) # Callback function to deal w/ results


### CAMERA FUNCTIONALITY ###

def jsob_to_image(js_object):
  # decode base64 image
  image_bytes = b64decode(js_object.split(',')[1])
  # convert bytes to numpy array
  img_array = np.frombuffer(image_bytes, dtype=np.uint8)
  # convert numpy array into OpenCV BGR
  frame = cv.imdecode(img_array, flags=1)

  return frame

def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);


      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);
      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="blue: red; font-weight: bold;">' +
          'click here to stop the video</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();
      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame() {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }
      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();



      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame():
  data = eval_js('stream_frame()')
  return data

### END OF CAMERA CODE ###

with PoseLandmarker.create_from_options(options) as landmarker:
  frame_timestamp_ms = 0
  video_stream()
  while True:
    frame_timestamp_ms += 1
    frame_js = video_frame()
    if not frame_js:
      break
    img = jsob_to_image(frame_js["img"])
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)

    landmarker.detect_async(mp_image, frame_timestamp_ms)

if record:
  out_video.release()